In [433]:
import numpy as np
import pyedflib

import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, ForgetDimension
from sklearn.model_selection import train_test_split, KFold
import random

# Choose EEG or EMG

In [507]:
# Choose if you want to look at EEG or EMG data

data_type = "EEG" # Does not have an effect yet, will be added later when processing anesthesia data
#data_type = "EMG"

In [558]:
subject = "293"


In [559]:
label_list = [1, 2, 3, 4, 5, 7]


# Import Data

In [560]:
# Labels

# Read data
label_data = pd.read_csv("Data/Labels_"+str(subject)+".csv")
labels = label_data["NAPS_Numeric"].iloc[1:] # The first label is NaN, so remove it

# Convert to list
labels = [int(label) for label in labels]


In [561]:
# EEG & EMG data

data = pd.read_csv("Data/EDF_as_CSV_"+str(subject)+"_complete.csv")

In [562]:
def preprocess_data(data, labels):

    data = data.iloc[2000:] # The first label is NaN... Therefore we have to remove the first 2000 values corresponding to the first label

    data.reset_index(drop=True, inplace=True)

    # Add labels to the main dataframe and Assign labels to data rows in segments 
    data['Label'] = np.nan
    
    for label_idx in range(len(labels) - 1):
        data.loc[2000 * label_idx : 2000 * (label_idx + 1) - 1, 'Label'] = labels[label_idx]
        
    # Drop rows where 'Label' is NaN
    data.dropna(subset=['Label'], inplace=True)

    # Drop columns containing 'Unnamed' in their column name
    columns_to_drop = data.filter(like='Unnamed').columns
    data.drop(columns=columns_to_drop, inplace=True)


    return data


In [563]:
data = preprocess_data(data, labels)

/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_22972/316733124.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = np.nan
/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_22972/316733124.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['Label'], inplace=True)
/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_22972/316733124.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

# Preprocessing

## Segmenting Data

In [564]:
def get_labelled_segments_and_indices(label_list, df, data_type, segment_size, step_size = 2):
    """
    Segments time-series data into EEG and EMG segments.

    Parameters:
    - label_list (list): List of all possible labels
    - data (DataFrame): The input dataframe containing the columns "Time", "EEG" and "EMG".
    - segment_size (float): The desired size of each segment in seconds.
    - step_size (float, optional): The step size of "Time" in milliseconds. Default is 2 millisecond.

    Returns:
    Tuple of two lists:
    - List of EEG segments.
    - List of EMG segments.
    """

    all_indices = {}

    all_segments = {}

    for label in label_list:

        segments = []
       
        label_df = df[df['Label'] == label]
        
        label_df.reset_index(drop=True, inplace=True)
        
        all_indices["Label_"+str(label)] = set(label_df.index*step_size // segment_size // 1000)
        
        n_segments_of_label = int((len(label_df.index))*step_size) // segment_size // 1000

        for segment_idx in range(n_segments_of_label):
            start_idx = int(segment_idx* segment_size*1000/step_size)
            end_idx = start_idx + int(segment_size*1000/step_size)
            segment = label_df[data_type].iloc[start_idx:end_idx]
            segments.append(list(segment))

        all_segments[label] = segments

    
    return all_segments, all_indices

In [565]:
# Segment the datass
segment_size = 4  # seconds


segments, all_indices_dict = get_labelled_segments_and_indices(label_list, data, data_type, segment_size)


# Compute Indices for Cross Validation and Final Test Set

In [566]:
# Choose test data set size for classification later (recommended: 0.2-0.3)

test_size = 0.2

In [567]:
# Choose number of folds for Cross Validation of the training/validation data

n_folds = 5

### Choose Train and Test Data Indices for Test Set and for Each Fold

In [568]:
def compute_test_set_indices(label_list, segments, test_size):
    total_data_length = 0

    for label in label_list:
        total_data_length += len(segments[label])

    train_indices_dict = {}
    test_indices_dict = {}
    all_indices_dict = {}


    for label in label_list:
        data_length = len(segments[label])
        indices = np.arange(data_length)

        # This weird reshaping will later allow us to handle train/test indices of the final test set
        # splitting and the Cross Validation sets the same way
        # Both of these dictionaries will only contain one key, 0, as if there would be only one fold
        train_indices_dict["Label_"+str(label)] = {}
        test_indices_dict["Label_"+str(label)] = {}

        _, _, _, _, train_indices_dict["Label_"+str(label)][0], test_indices_dict["Label_"+str(label)][0] = train_test_split(segments[label], [label]*data_length, indices, test_size=test_size, random_state=32)

        all_indices_dict["Label_"+str(label)] = np.concatenate((train_indices_dict["Label_"+str(label)][0], test_indices_dict["Label_"+str(label)][0]))


    return train_indices_dict, test_indices_dict, all_indices_dict


# Compute indices for final test set
train_indices_dict, test_indices_dict, all_indices_dict = compute_test_set_indices(label_list, segments, test_size)

In [569]:
def choose_train_and_test_indices(all_indices_dict, segments, test_size):
    train_indices_dict = {}
    test_indices_dict = {}

    # Iterate through each label and perform train/test split
    for label, indices in all_indices_dict.items():


        # Perform train/test split
        train_indices, test_indices = train_test_split(all_indices_dict[label], test_size=test_size, random_state=32)


        # Store the split indices
        train_indices_dict[label] = train_indices
        test_indices_dict[label] = test_indices


    return train_indices_dict, test_indices_dict


# Choose train and test indices
train_indices_dict, test_indices_dict = choose_train_and_test_indices(all_indices_dict, segments, test_size)

In [570]:
def compute_train_and_validation_set_indices_for_cross_validation(label_list, segments, train_indices_dict, n_folds):
    
    # Initialize Cross Validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    train_indices_dict_for_folds = {}
    validation_indices_dict_for_folds = {}
    
    for label in label_list:
        label_key = "Label_" + str(label)
        
        # Initialize dictionary for each label
        train_indices_dict_for_folds[label_key] = {}
        validation_indices_dict_for_folds[label_key] = {}
        
        label_indices = train_indices_dict[label_key]
        
        # Use KFold only if there are enough samples for cross-validation
        if len(label_indices) >= 5:
            for fold, (train_indices, validation_indices) in enumerate(kf.split(label_indices)):
                train_indices_dict_for_folds[label_key][f"Fold_{fold}"] = label_indices[train_indices]
                validation_indices_dict_for_folds[label_key][f"Fold_{fold}"] = label_indices[validation_indices]
        else:
            # Handle case where fewer than 5 samples are available
            for fold in range(len(label_indices)):
                train_indices_dict_for_folds[label_key][f"Fold_{fold}"] = [label_indices[fold]]
                validation_indices_dict_for_folds[label_key][f"Fold_{fold}"] = [label_indices[fold]]

            # Fill in remaining folds with empty lists if needed
            for remaining_fold in range(len(label_indices), n_folds):
                train_indices_dict_for_folds[label_key][f"Fold_{remaining_fold}"] = []
                validation_indices_dict_for_folds[label_key][f"Fold_{remaining_fold}"] = []
    
    return train_indices_dict_for_folds, validation_indices_dict_for_folds


train_indices_dict_for_folds, validation_indices_dict_for_folds = compute_train_and_validation_set_indices_for_cross_validation(label_list,
                                                                                    segments, train_indices_dict, n_folds)

In [571]:
# Save all indices

# Train indices
np.save("Train_Test_Splitting/"+str(subject)+"/Train_Indices_All_Labels_All_Folds.npy",\
    np.array(train_indices_dict_for_folds, dtype=object), allow_pickle=True)

# Validation indices
np.save("Train_Test_Splitting/"+str(subject)+"/Validation_Indices_All_Labels_All_Folds.npy",\
    np.array(validation_indices_dict_for_folds, dtype=object), allow_pickle=True)

# Final Test set indices
np.save("Train_Test_Splitting/"+str(subject)+"/Final_Test_Set_Indices_All_Labels.npy",\
    np.array(test_indices_dict, dtype=object), allow_pickle=True)



# Persistence Diagrams

## Finding the optimal embedding dimension and time delay

There are two techniques that can be used to determine these parameters automatically:
- Mutual information to determine the time delay
- False nearest neighbours to determine the embedding dimension

In [ ]:
# Initialise the embedding
max_embedding_dimension = 30
max_time_delay = 30
stride = 10

embedder = SingleTakensEmbedding(
    parameters_type="search",
    time_delay=max_time_delay,
    dimension=max_embedding_dimension,
    stride=stride,
)

In [ ]:
def find_optimal_parameters(embedder, segments, max_index, iterations = 8):
    """
    Finds (approximate) optimal embedding parameters by averaging optimal parameters of random segments.

    Parameters:
    - embedder (object): defined by SingleTakensEmbedding() or similar
    - segments (list of lists): Complete EEG/EMG segments
    - max_index (int): How many segments there are
    - iteratiors (int): How many random indices to sample

    Returns:
    Tuple of two floats:
    - Average optimal embedding dimension
    - Average optimal time delay
    """


    optimal_embeddings_dimensions = []
    optimal_time_delays = []
    
    for _ in range(iterations):
        random_index = random.randint(0, max_index)
        embedding = embedder.fit_transform(segments[random_index])
        
         # append optimal embedding dimension for this segment
        optimal_embeddings_dimensions.append(embedder.dimension_)

        # append optimal time delay for this segment
        optimal_time_delays.append(embedder.time_delay_)

        print("The optimal embedding dimension is " + str(np.mean(optimal_embeddings_dimensions)) + 
              " and the optimal time delay is " + str(np.mean(optimal_time_delays)))
        
        return int(np.mean(optimal_embeddings_dimensions)), int(np.mean(optimal_time_delays))

In [ ]:
# Compute optimal embedding parameters
    
embedding_dimension, embedding_time_delay = find_optimal_parameters(embedder, all_segments, len(all_segments), iterations = 8)

## Creating Persistence Diagrams

In [ ]:
# Setting parameters for point cloud embeddings

#embedding_dimension= 3 # for data exploration

embedder = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay, # computed above
    dimension=embedding_dimension, # computed above
    stride=stride,
)

In [ ]:
# We will look at 0, 1 and 2 dimensional holes
homology_dimensions = [0, 1, 2]

# We will use a Vietoris Rips filtrations
persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=10
)

### Computing Points Clouds and Persistence Diagrams

In [ ]:
def compute_embeddings_and_diagrams(segments, time_delay_embeddings, persistence_diagrams, all_indices_dict, label):

    time_delay_embeddings["Label_"+str(label)] = []
    persistence_diagrams["Label_"+str(label)] = []

    # Compute embeddings and diagrams for the complete data
    for diagram_idx in range(len(segments[label])):
        time_delay_embeddings["Label_"+str(label)].append(embedder.fit_transform(segments[label][diagram_idx])[None, :, :])
        persistence_diagrams["Label_"+str(label)].append(persistence.fit_transform(time_delay_embeddings["Label_"+str(label)][diagram_idx]))
    
    return time_delay_embeddings, persistence_diagrams

In [ ]:
# Compute embeddings and persistence diagrams for the complete data

time_delay_embeddings = {}
persistence_diagrams = {}

for label in label_list:
    time_delay_embeddings, persistence_diagrams = compute_embeddings_and_diagrams(segments, time_delay_embeddings, persistence_diagrams, all_indices_dict, label)

## Save persistence diagrams and embeddings

In [ ]:
# Persistence diagrams
np.save('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/'+str(data_type)+'/Persistence_Diagrams_All_Labels.npy', \
            np.array(persistence_diagrams, dtype=object), allow_pickle=True)

# Embeddings
np.save('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/'+str(data_type)+'/Embeddings_All_Labels'+\
            '_Embedding_Dim'+str(embedding_dimension)+'.npy', np.array(time_delay_embeddings, dtype=object), allow_pickle=True)